# 0. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import math

# 1. Construct dataFrame about NCM and its adjacent pores

In [76]:
d = pd.read_csv('/home/sbml/Desktop/0.solid_electrolyte/id_240728/coating_ncm_id.csv')
id_list = d['id'].tolist()

In [77]:
def pore_ind(data):
    file = data['file'].tolist()[0]
    df = pd.read_csv(file).drop(['Unnamed: 0', 'center'], axis = 1)

    cx_l, cy_l = [], []
    frame_l, cordi_l = [], []
    pore_index = []
    col_list = df.columns.tolist()
    frame_list = df['#frame'].tolist()
    for col in col_list[1:]:
        ncm_list = df[col].tolist()
        
        for i in range(len(frame_list)):
            if  np.isnan(ncm_list[i]) != True:
                target_frame = int(frame_list[i])
                target_ncm = int(ncm_list[i])

                d_target = pd.read_csv('/home/sbml/Desktop/0.solid_electrolyte/id_240728/1.coating/ncm_pore_id_cordi/frame' + str(target_frame) + '.csv')
                ncm = np.array(eval( d_target['ncm_cordi'].tolist()[target_ncm]))
                
                pore = eval( d_target['pore'].tolist()[target_ncm])
                pore_ind = eval(d_target['pore_ind'].tolist()[target_ncm])
                #print(pore_ind)
                for pp in range(len(pore)):
                    p = pore[pp]
                    
                    for p1 in p:
                        p2 = np.array(p1)
                        M = cv.moments(p2)
                        cx = int(M['m10']/M['m00'])
                        cy = int(M['m01']/M['m00'])
                        cx_l.append(cx)
                        cy_l.append(cy)
                        frame_l.append(target_frame)
                        xx = p2[:,0]
                        yy = p2[:,1]
                        xx_list = []
                        for xxx in range(len(xx)):
                            xx_list.append([xx[xxx], yy[xxx]])
                        cordi_l.append(xx_list)  
                        pore_index.append(pore_ind[pp])
                        
                    
    return cx_l, cy_l, frame_l, cordi_l, pore_index

In [78]:
cen_x, cen_y = [],[]
frame_list, cordination_list = [], []
index_list = []
for f in id_list:
    data = d[d['id'] == f]
    cx_l, cy_l, frame_l, cordi_l, pore_index = pore_ind(data)
    index_list.append(pore_index)
    cen_x.append(cx_l)
    cen_y.append(cy_l)
    frame_list.append(frame_l)
    cordination_list.append(cordi_l)

In [79]:
d_cen = pd.DataFrame({})
d_cen['ncm_id'] = id_list
d_cen['frame_list'] = frame_list
d_cen['cordination_list'] = cordination_list
d_cen['cen_x'] = cen_x
d_cen['cen_y'] = cen_y
d_cen['pore_index'] = index_list
#d_cen.to_csv('/home/sbml/Desktop/0.solid_electrolyte/id_240728/0.bare/pore_listup_240722.csv')
d_cen

,ncm_id,frame_list,cordination_list,cen_x,cen_y,pore_index
0,0,"[100, 101, 104, 106, 106, 106, 107, 107, 107, ...","[[[493, 1112], [493, 1113], [494, 1113], [495,...","[507, 501, 492, 499, 492, 513, 500, 492, 513, ...","[1131, 1141, 1152, 1120, 1148, 1130, 1127, 114...","[0, 0, 0, 3, 4, 5, 2, 3, 4, 3, 4, 5, 2, 2, 2, ..."
1,1,"[111, 112, 114, 115, 116, 117, 119, 120, 123, ...","[[[328, 67], [327, 68], [326, 69], [325, 70], ...","[325, 487, 480, 479, 479, 479, 472, 474, 488, ...","[72, 76, 90, 90, 89, 90, 100, 98, 80, 81, 84, ...","[266, 238, 251, 275, 279, 264, 245, 238, 270, ..."
2,2,"[102, 103, 104, 104, 105, 109, 109, 110, 110, ...","[[[138, 566], [138, 567], [138, 568], [137, 56...","[137, 137, 125, 141, 141, 127, 147, 117, 127, ...","[578, 577, 588, 574, 572, 586, 577, 576, 586, ...","[133, 135, 141, 142, 135, 147, 148, 150, 151, ..."
3,3,"[102, 103, 103, 103, 103, 104, 104, 104, 104, ...","[[[481, 388], [482, 389], [481, 390], [482, 39...","[485, 489, 478, 478, 489, 477, 490, 477, 490, ...","[396, 397, 397, 397, 397, 399, 397, 399, 397, ...","[180, 179, 180, 181, 181, 193, 193, 194, 195, ..."
4,4,"[104, 105, 105, 106, 107, 108, 108, 108, 110, ...","[[[438, 627], [437, 628], [436, 628], [435, 62...","[434, 422, 434, 435, 434, 404, 425, 434, 402, ...","[630, 642, 630, 631, 632, 639, 645, 632, 639, ...","[120, 111, 112, 128, 138, 117, 118, 119, 137, ..."
...,...,...,...,...,...,...
495,495,"[93, 94, 98, 101, 102, 103, 106, 107, 107, 109...","[[[441, 689], [441, 690], [441, 691], [442, 69...","[443, 443, 427, 444, 445, 445, 429, 449, 429, ...","[693, 694, 687, 657, 659, 659, 664, 663, 664, ...","[125, 117, 102, 89, 109, 111, 120, 118, 119, 1..."
496,496,"[94, 95, 96, 97, 97, 98, 99, 117, 118, 119]","[[[702, 316], [701, 317], [700, 317], [699, 31...","[705, 705, 689, 690, 704, 679, 678, 679, 679, ...","[322, 321, 324, 324, 319, 320, 319, 311, 311, ...","[226, 219, 235, 225, 226, 221, 214, 202, 188, ..."
497,497,"[99, 117, 118, 123, 124, 125, 126, 167, 167, 1...","[[[687, 926], [688, 926], [689, 926], [690, 92...","[698, 689, 692, 664, 674, 675, 674, 663, 650, ...","[929, 958, 932, 960, 939, 969, 969, 918, 925, ...","[49, 43, 37, 50, 52, 47, 44, 51, 52, 42, 43, 6..."
498,498,"[99, 102, 103, 106, 108, 109, 110, 111, 111, 1...","[[[202, 981], [202, 982], [202, 983], [202, 98...","[199, 204, 204, 201, 200, 200, 200, 220, 225, ...","[985, 988, 988, 966, 968, 967, 967, 988, 971, ...","[37, 34, 37, 36, 31, 35, 30, 31, 32, 28, 29, 3..."


In [3]:
l = []
l_cri = []
l0_list = []
for i in range(len(cen_x)):
    l.append(len(cen_x[i]))    

    l0 = []
    for j in range(1,len(cen_x[i])):
        t1 = np.array([cen_x[i][j], cen_y[i][j], frame_list[i][j]])
        t2 = np.array([cen_x[i][j-1], cen_y[i][j-1], frame_list[i][j]])
        l0.append(math.dist(t1, t2))
    
    if len(cen_x[i]) <= 2: 
        l_cri.append('pass')
        l0_list.append([])
        
    elif len(cen_x[i]) > 2: 
        l0 = np.array(l0)
        l0 = np.where(l0 > 2)[0].tolist()
        l0_list.append(l0)
        if l0 == []:
            l_cri.append('pass')
        elif l0 != [] :
            l_cri.append('check')
            
d_cen['cri'] = l_cri
d_cen

In [81]:
p_cate_l = []
cnt = 0
for i in range(len(l0_list)):
    p_cate = []
    leng = len(frame_list[i])
    a = l0_list[i] + [leng]
    
    a1 = [a[i] - a[i-1] for i in range(1, len(a))]
    a1 = [a[0]] + a1
    if l0_list[i] == []:
        p_cate= p_cate +  [cnt for k in range(leng)]
        cnt += 1
    elif l0_list[i] != []: 
        for j in a1:

            p_cate = p_cate + [cnt] * j
            cnt += 1
    p_cate_l.append(p_cate)     
d_cen['pore_category'] = p_cate_l

In [83]:
d_cen.to_csv('/home/sbml/Desktop/0.solid_electrolyte/id_240729/try1_xyz/1.coating/pore_listup_240729.csv')

In [84]:
final_ind = max(p_cate_l[-1])
a = p_cate_l

9150

In [86]:
id_fin = []
frame_fin = []
cordinate_fin = []
p_index_fin = []
ncm_index_fin = []
for i in range(len(a)):
    id_fin = id_fin + p_cate_l[i] 
    frame_fin = frame_fin + frame_list[i]
    p_index_fin = p_index_fin + index_list[i]
    cordinate_fin = cordinate_fin + cordination_list[i]
    ncm_index_fin = ncm_index_fin + [id_list[i]]*len(cordination_list[i])

In [89]:
df = pd.DataFrame({})
df['Pore_id'] = id_fin
df['#frame'] = frame_fin
df['pore_index_per_frame'] = p_index_fin
df['pore_cordinate'] = cordinate_fin
df['ncm_index_per_frame'] = ncm_index_fin
df

,Pore_id,#frame,pore_index_per_frame,pore_cordinate,ncm_index_per_frame
0,1,100,0,"[[493, 1112], [493, 1113], [494, 1113], [495, ...",0
1,2,101,0,"[[493, 1112], [493, 1113], [493, 1114], [494, ...",0
2,3,104,0,"[[491, 1142], [491, 1143], [491, 1144], [491, ...",0
3,4,106,3,"[[497, 1103], [496, 1104], [495, 1105], [495, ...",0
4,5,106,4,"[[496, 1130], [495, 1131], [494, 1131], [493, ...",0
...,...,...,...,...,...
11350,9149,114,7,"[[159, 1068], [159, 1069], [159, 1070], [159, ...",499
11351,9149,115,10,"[[170, 1072], [169, 1073], [168, 1073], [167, ...",499
11352,9150,116,14,"[[170, 1072], [169, 1073], [168, 1073], [167, ...",499
11353,9150,117,13,"[[154, 1073], [154, 1074], [154, 1075], [154, ...",499


In [90]:
f = frame_fin
adjacent_id_l = []

for i in range(len(f)):
    d = df[df['#frame'] == i]
    p_filter = d['pore_index_per_frame'].tolist()
    p_filter = list(set(p_filter))

    for j in p_filter:
        t = d[d['pore_index_per_frame'] == j]  
        n_filter = t['ncm_index_per_frame'].tolist()
        n_filter = list(set(n_filter))
      
        if len(n_filter) >= 2:
            adjacent_id_l.append(t['Pore_id'].tolist())

In [91]:
for i in adjacent_id_l[0]:
    t = df[df['Pore_id'] == i]
    display(t)

,Pore_id,#frame,pore_index_per_frame,pore_cordinate,ncm_index_per_frame
4610,3728,46,168,"[[699, 428], [699, 429], [700, 430], [701, 431...",184


,Pore_id,#frame,pore_index_per_frame,pore_cordinate,ncm_index_per_frame
7866,6362,46,168,"[[699, 428], [699, 429], [700, 430], [701, 431...",331


In [92]:
adj_fin = []
for i in id_fin:
    ad = []
    for j in adjacent_id_l:
        if i in j:
            ad.append(j)
    adj_fin.append(ad)

In [93]:
df['adj_id'] = adj_fin

In [94]:
df

,Pore_id,#frame,pore_index_per_frame,pore_cordinate,ncm_index_per_frame,adj_id
0,1,100,0,"[[493, 1112], [493, 1113], [494, 1113], [495, ...",0,[]
1,2,101,0,"[[493, 1112], [493, 1113], [493, 1114], [494, ...",0,[]
2,3,104,0,"[[491, 1142], [491, 1143], [491, 1144], [491, ...",0,[]
3,4,106,3,"[[497, 1103], [496, 1104], [495, 1105], [495, ...",0,[]
4,5,106,4,"[[496, 1130], [495, 1131], [494, 1131], [493, ...",0,[]
...,...,...,...,...,...,...
11350,9149,114,7,"[[159, 1068], [159, 1069], [159, 1070], [159, ...",499,[]
11351,9149,115,10,"[[170, 1072], [169, 1073], [168, 1073], [167, ...",499,[]
11352,9150,116,14,"[[170, 1072], [169, 1073], [168, 1073], [167, ...",499,"[[404, 9150]]"
11353,9150,117,13,"[[154, 1073], [154, 1074], [154, 1075], [154, ...",499,"[[404, 9150]]"


In [95]:
for i in range(final_ind+1):
    d = df[df['Pore_id'] == i]
    d.to_csv('/home/sbml/Desktop/0.solid_electrolyte/240729_new_pore_index/1.coating/pore_particle_2/' + str(i) + '.csv')

In [96]:
d = pd.read_csv('/home/sbml/Desktop/0.solid_electrolyte/240729_new_pore_index/1.coating/pore_particle_2/' + str(i) + '.csv')
d

,Unnamed: 0,Pore_id,#frame,pore_index_per_frame,pore_cordinate,ncm_index_per_frame,adj_id
0,11352,9150,116,14,"[[170, 1072], [169, 1073], [168, 1073], [167, ...",499,"[[404, 9150]]"
1,11353,9150,117,13,"[[154, 1073], [154, 1074], [154, 1075], [154, ...",499,"[[404, 9150]]"
2,11354,9150,118,12,"[[160, 1074], [160, 1075], [160, 1076], [159, ...",499,"[[404, 9150]]"
